In [1]:
from gensim.models import Doc2Vec
from tensorflow.keras import models
from trec.trectopics import TrecTopics
from arcs.data_generator import DataGenerator

Load all necessary models

In [2]:
d2v_model_file = "f:/Models/Lemma_DBOW_2_doc2vec_trec_d1000_n5_w8_mc20_t12_e10_dbow.model"
retrieval_sbm_model = "f:/Models/NN/sbm_predict"
topics_path = "f:/Runs/topics"

In [3]:
d2v = Doc2Vec.load(d2v_model_file)
sbm = models.load_model(retrieval_sbm_model)
topics = TrecTopics(topics_path=topics_path)

# vectorize TREC queries
topics.vectorize(vocab_dict=d2v.wv.vocab,
                 include_title=True, include_desc=False, include_narr=False)

Create data generator for retrieval SBM model prediction

In [6]:
topic_no = 51   # <===== target query number

docvecs = d2v.docvecs.vectors_docs
targer_vecs = list(topics.get_topic_vector(topic_no)) * len(docvecs)

data_feeder = DataGenerator(docvecs, targer_vecs, shuffle=False, batch_size=64)

MemoryError: 

Compute cosine similarity of all document with particular topic query

In [ ]:
cossim = sbm.predict(x=data_feeder, verbose=1, use_multiprocessing=False)

Sort document by descending order of cosine similarity

In [ ]:
docno_list = d2v.docvecs.index2entity
result = dict(zip(docno_list, cossim))

sorted_result = sorted(result.items(), key=lambda x: x[1], reverse=True)